# Prepare versions of NWM Reanalysis dataset in Zarr format

* Save a subset of NWM reanalysis covering a single HUC2 for testing purposes. The size is supposed to be not so small that it is trivial, but not so big that things take a really long time to run. 
* Rechunk this subset so we have versions with rectangular chunks, row chunks, and column chunks.|

## Setup

In [15]:
import json
from os.path import basename, join
import os 
import shutil
from os import makedirs

from dask.distributed import Client
import xarray as xr
import fsspec
from rechunker import rechunk
import s3fs
import fsspec

%matplotlib inline

def get_json(uri):
    with fsspec.open(uri) as fd:
        return json.load(fd)

client = Client()

/opt/conda/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33221 instead
  warnings.warn(


In [10]:
# Set various URIs.

# The CHRTOUT data from the NWM Retrospective Zarr 2.1 dataset
# This has "Streamflow values at points associated with flow lines" 
# See https://registry.opendata.aws/nwm-archive/
nwm_uri = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'

# URI of JSON file with COMIDS to use for creating subset of NWM. 
# This was produced by save_huc2_comids.ipynb.
comid_uri = 's3://azavea-noaa-hydro-data/noaa/huc2-comids.json'

# This root URI should be updated for each run of this notebook.
out_root_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/07-05-2022a'
nwm_subset_uri = join(out_root_uri, 'nwm-subset.zarr')
rechunk_tmp_uri = join(out_root_uri, 'rechunk-tmp.zarr')
square_chunk_uri = join(out_root_uri, 'square-chunk.zarr')

# Clear old contents of output directories.
def rm(x):
    try:
        s3fs.S3FileSystem().rm(x, recursive=True)
    except FileNotFoundError:
        pass

rm(out_root_uri)

## Save subet of NWM

In [13]:
ds = xr.open_zarr(fsspec.get_mapper(nwm_uri, anon=True, requester_pays=True))

comids = get_json(comid_uri)['comids']

avail_comids = list(set(ds.feature_id.values).intersection(set(comids)))
# Need the following line to sort the comids or will get the following warning.
# PerformanceWarning: Slicing with an out-of-order index is generating 736 times more chunks
avail_comids.sort()
print(
    f'There are {len(comids)} reaches in the HUC and {len(avail_comids)} of those are in NWM.')

# TODO: set this to a longer range, maybe 10 years?
time_range = slice('01-01-1990', '01-02-1990')

There are 137288 reaches in the HUC and 122256 of those are in NWM.


In [14]:
sub_ds = ds.sel(time=time_range, feature_id=avail_comids)

# Fails without this block. See https://github.com/pydata/xarray/issues/5219 
def remove_chunks_encoding(ds):
    for var in list(ds.keys()) + list(ds.coords):
        if 'chunks' in ds[var].encoding:
            del ds[var].encoding['chunks']
    return ds

sub_ds = remove_chunks_encoding(sub_ds)
sub_ds

<xarray.Dataset>
Dimensions:     (feature_id: 122256, time: 48)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(122256,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 1990-01-02T23:00:00
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(48, 445), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(48, 445), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [23]:
sub_ds.streamflow

<xarray.DataArray 'streamflow' (time: 48, feature_id: 122256)>
dask.array<getitem, shape=(48, 122256), dtype=float64, chunksize=(48, 24143), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(122256,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 1990-01-02T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [15]:
# Fails with the following error if chunk(<int>) isn't called.
# ValueError: Zarr requires uniform chunk sizes except for final chunk. Variable named 'streamflow' has incompatible dask chunks: ((48,), (445, 5348, 802, 2192, 3915, 10509, 12946, 3414, 7504, 87, 13247, 6817, 24143, 2, 11164, 1156, 3632, 499, 1518, 1666, 1633, 8, 3, 4965, 3147, 723, 771)). Consider rechunking using `chunk()`.
sub_ds = remove_chunks_encoding(sub_ds.chunk(1000))

In [8]:
sub_ds.streamflow

<xarray.DataArray 'streamflow' (time: 48, feature_id: 122256)>
dask.array<rechunk-merge, shape=(48, 122256), dtype=float64, chunksize=(48, 1000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(1000,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(1000,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 1990-01-02T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [16]:
sub_ds.to_zarr(nwm_subset_uri)

In [27]:
sub_ds.chunks

Frozen({'feature_id': (1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 256), 'time': (48,)})

## Save rechunked versions of subset of NWM.

In [12]:
def _rechunk(target_chunks, output_uri):
    max_mem = '1GB'
    rm(rechunk_tmp_uri)
    ds = xr.open_zarr(nwm_subset_uri)

    # Note, if you get a ContainsArrayError, you probably need to delete temp_store and target_store first.
    # See https://github.com/pangeo-data/rechunker/issues/78

    targ_store = fsspec.get_mapper(output_uri)
    temp_store = fsspec.get_mapper(rechunk_tmp_uri)
    rechunk_plan = rechunk(ds, target_chunks, max_mem, targ_store, temp_store=temp_store)
    rechunk_plan.execute()

In [13]:
# Each chunk is a square.
target_chunks = {'feature_id': 300, 'time': 300}
_rechunk(target_chunks, square_chunk_uri)
rechunked_ds = xr.open_zarr(square_chunk_uri)
rechunked_ds.chunks

/tmp/ipykernel_9555/1979998546.py:4: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  rechunked_ds = xr.open_zarr(square_chunk_uri)


Frozen({'feature_id': (300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 30